In [ ]:
from fractions import Fraction
import copy

def invert_matrix(B):
    n = len(B)
    aug = []
    for i in range(n):
        row = []
        for j in range(n):
            row.append(B[i][j])
        for j in range(n):
            row.append(Fraction(1) if i == j else Fraction(0))
        aug.append(row)
    
    for col in range(n):
        pivot_row = col
        for r in range(col + 1, n):
            if abs(aug[r][col]) > abs(aug[pivot_row][col]):
                pivot_row = r
        aug[col], aug[pivot_row] = aug[pivot_row], aug[col]
        pivot_val = aug[col][col]
        if pivot_val == 0:
            raise ValueError("Matrix is singular.")
        for j in range(2 * n):
            aug[col][j] /= pivot_val
        for i in range(n):
            if i == col:
                continue
            factor = aug[i][col]
            for j in range(2 * n):
                aug[i][j] -= factor * aug[col][j]
    
    inv = []
    for i in range(n):
        inv.append(aug[i][n:])
    return inv

def mat_mult(A, B):
    n = len(A)
    p = len(B)
    m = len(B[0])
    C = [[Fraction(0) for _ in range(m)] for _ in range(n)]
    for i in range(n):
        for k in range(p):
            if A[i][k] != 0:
                for j in range(m):
                    C[i][j] += A[i][k] * B[k][j]
    return C

class SimplexMethod:
    def __init__(self, A, b, c, base_indexes):
        self.m = len(A)
        self.n = len(A[0])
        self.A = [[Fraction(elem) for elem in row] for row in A]
        self.b = [Fraction(elem) for elem in b]
        self.c = [Fraction(elem) for elem in c]
        self.base_indexes = base_indexes
        self.table = None
        self.build_initial_tableau()
        
    def build_initial_tableau(self):
        B = [[self.A[i][j] for j in self.base_indexes] for i in range(self.m)]
        invB = invert_matrix(B)
        T_constraints = mat_mult(invB, self.A)
        b_new = mat_mult(invB, [[bi] for bi in self.b])
        base_c = [self.c[j] for j in self.base_indexes]
        delta0 = sum(base_c[i] * b_new[i][0] for i in range(self.m))
        delta_j = []
        for j in range(self.n):
            z_j = sum(base_c[i] * T_constraints[i][j] for i in range(self.m))
            delta_j.append(z_j - self.c[j])
        
        self.table = [[Fraction(0) for _ in range(self.n + 1)] for _ in range(self.m + 1)]
        for i in range(self.m):
            for j in range(self.n):
                self.table[i][j] = T_constraints[i][j]
            self.table[i][self.n] = b_new[i][0]
        for j in range(self.n):
            self.table[self.m][j] = delta_j[j]
        self.table[self.m][self.n] = delta0
        
    def solve(self):
        iteration = 0
        while True:
            print(f"Iteration {iteration}:")
            self.print_table()
            if all(self.table[self.m][j] >= 0 for j in range(self.n)):
                print("Optimal solution reached.")
                break
            j0 = min(range(self.n), key=lambda j: self.table[self.m][j])
            if self.table[self.m][j0] >= 0:
                break
            ratios = []
            for i in range(self.m):
                if self.table[i][j0] > 0:
                    ratio = self.table[i][self.n] / self.table[i][j0]
                    ratios.append((ratio, i))
            if not ratios:
                print("Problem is unbounded.")
                return
            theta, i0 = min(ratios, key=lambda x: x[0])
            self.base_indexes[i0] = j0
            pivot = self.table[i0][j0]
            for j in range(self.n + 1):
                self.table[i0][j] /= pivot
            for i in range(self.m + 1):
                if i == i0:
                    continue
                factor = self.table[i][j0]
                for j in range(self.n + 1):
                    self.table[i][j] -= factor * self.table[i0][j]
            iteration += 1
            
    def print_table(self):
        header = [f"x{j}" for j in range(self.n)] + ["b"]
        print("\t".join(header))
        for i in range(self.m):
            row = [str(self.table[i][j]) for j in range(self.n + 1)]
            print(f"x{self.base_indexes[i]}\t" + "\t".join(row))
        row = [str(self.table[self.m][j]) for j in range(self.n + 1)]
        print("Δ\t" + "\t".join(row))
        print()

# Example usage from the lecture
if __name__ == "__main__":
    # Example problem: maximize 3x1 + 2x2 with constraints
    A = [
        [4, 5, 1, 0, 0],
        [2, 7, 0, 1, 0],
        [8, 3, 0, 0, 1]
    ]
    b = [58, 65, 90]
    #f = 3x1+4x2
    c = [3, 4, 0, 0, 0]
    base_indexes = [2, 3, 4]  # Initial basis: x3, x4, x5
    simplex = SimplexMethod(A, b, c, base_indexes)
    simplex.solve()

Iteration 0:
x0	x1	x2	x3	x4	b
x2	4	5	1	0	0	58
x3	2	7	0	1	0	65
x4	8	3	0	0	1	90
Δ	-3	-4	0	0	0	0

Iteration 1:
x0	x1	x2	x3	x4	b
x2	18/7	0	1	-5/7	0	81/7
x1	2/7	1	0	1/7	0	65/7
x4	50/7	0	0	-3/7	1	435/7
Δ	-13/7	0	0	4/7	0	260/7

Iteration 2:
x0	x1	x2	x3	x4	b
x0	1	0	7/18	-5/18	0	9/2
x1	0	1	-1/9	2/9	0	8
x4	0	0	-25/9	14/9	1	30
Δ	0	0	13/18	1/18	0	91/2

Optimal solution reached.


In [1]:
from fractions import Fraction
import numpy as np
from IPython.display import display, HTML

def print_simplex_table(tableau, basis, iteration):
    """Красиво выводит симплекс-таблицу в Jupyter Notebook"""
    headers = [f'x{i}' for i in range(1, len(tableau[0]))] + ['b']
    rows = []
    
    # Добавляем строки для базисных переменных
    for i in range(len(tableau)-1):
        row_data = [str(tableau[i][j]) for j in range(len(tableau[0]))]
        rows.append([f'x{basis[i]}'] + row_data)
    
    # Добавляем строку оценок
    delta_row = [str(tableau[-1][j]) for j in range(len(tableau[0]))]
    rows.append(['Δ'] + delta_row)
    
    # Создаем HTML таблицу
    html = f'<h3>Итерация {iteration}</h3><table border="1">'
    html += '<tr><th>Базис</th>' + ''.join(f'<th>{h}</th>' for h in headers) + '</tr>'
    
    for row in rows:
        html += '<tr><td>' + row[0] + '</td>' + ''.join(f'<td>{val}</td>' for val in row[1:]) + '</tr>'
    
    html += '</table>'
    display(HTML(html))

def simplex_method(A, b, c, initial_basis):
    """
    Реализация симплекс-метода для задачи максимизации в канонической форме
    
    Параметры:
    A - матрица ограничений (list of lists)
    b - вектор правых частей (list)
    c - коэффициенты целевой функции (list)
    initial_basis - начальный базис (list индексов базисных переменных)
    """
    # Преобразуем все входные данные в дроби
    m = len(A)
    n = len(A[0])
    
    A_frac = [[Fraction(elem) for elem in row] for row in A]
    b_frac = [Fraction(elem) for elem in b]
    c_frac = [Fraction(elem) for elem in c]
    
    # Текущий базис и таблица
    basis = initial_basis.copy()
    tableau = []
    
    # Строим начальную симплекс-таблицу
    for i in range(m):
        row = []
        for j in range(n):
            row.append(A_frac[i][j])
        row.append(b_frac[i])
        tableau.append(row)
    
    # Добавляем строку оценок
    delta_row = []
    for j in range(n):
        delta = Fraction(0)
        for i in range(m):
            delta += c_frac[basis[i]] * A_frac[i][j]
        delta -= c_frac[j]
        delta_row.append(delta)
    
    delta0 = Fraction(0)
    for i in range(m):
        delta0 += c_frac[basis[i]] * b_frac[i]
    
    delta_row.append(delta0)
    tableau.append(delta_row)
    
    iteration = 0
    print_simplex_table(tableau, basis, iteration)
    
    # Основной цикл симплекс-метода
    while True:
        # Проверка оптимальности (все оценки неотрицательны)
        if all(tableau[-1][j] >= 0 for j in range(n)):
            print("Оптимальное решение найдено!")
            break
        
        # Выбор разрешающего столбца (наименьшая отрицательная оценка)
        min_delta = Fraction(0)
        pivot_col = -1
        for j in range(n):
            if tableau[-1][j] < min_delta:
                min_delta = tableau[-1][j]
                pivot_col = j
        
        if pivot_col == -1:
            print("Не удалось найти разрешающий столбец")
            break
        
        # Проверка на неограниченность
        if all(tableau[i][pivot_col] <= 0 for i in range(m)):
            print("Задача неограниченна!")
            break
        
        # Выбор разрешающей строки (минимальное симплексное отношение)
        min_ratio = None
        pivot_row = -1
        for i in range(m):
            if tableau[i][pivot_col] > 0:
                ratio = tableau[i][-1] / tableau[i][pivot_col]
                if min_ratio is None or ratio < min_ratio:
                    min_ratio = ratio
                    pivot_row = i
        
        if pivot_row == -1:
            print("Не удалось найти разрешающую строку")
            break
        
        # Обновляем базис
        basis[pivot_row] = pivot_col
        
        # Преобразование Жордана-Гаусса
        pivot_element = tableau[pivot_row][pivot_col]
        
        # Делим разрешающую строку на разрешающий элемент
        for j in range(n + 1):
            tableau[pivot_row][j] /= pivot_element
        
        # Обнуляем разрешающий столбец в других строках
        for i in range(m + 1):
            if i == pivot_row:
                continue
            factor = tableau[i][pivot_col]
            for j in range(n + 1):
                tableau[i][j] -= factor * tableau[pivot_row][j]
        
        iteration += 1
        print_simplex_table(tableau, basis, iteration)
    
    # Формируем результат
    solution = [Fraction(0)] * n
    for i in range(m):
        if basis[i] < n:
            solution[basis[i]] = tableau[i][-1]
    
    optimal_value = tableau[-1][-1]
    
    return solution, optimal_value

# Пример использования (задача из лекции)
if __name__ == "__main__":
    # Максимизировать: 3x1 + 2x2
    # При ограничениях:
    # 2x1 + x2 ≤ 18
    # 2x1 + 3x2 ≤ 42
    # 3x1 + x2 ≤ 24
    # x1, x2 ≥ 0
    
    # Каноническая форма с добавлением slack-переменных:
    A = [
        [4, 5, 1, 0, 0],
        [2, 7, 0, 1, 0],
        [8, 3, 0, 0, 1]
    ]
    b = [58, 65, 90]
    #f = 3x1+4x2
    c = [3, 4, 0, 0, 0]
    initial_basis = [2, 3, 4]  # Начальный базис: x3, x4, x5
    
    solution, optimal_value = simplex_method(A, b, c, initial_basis)
    
    print("\nРезультат:")
    for i, val in enumerate(solution):
        print(f"x{i+1} = {val}")
    print(f"Оптимальное значение целевой функции: {optimal_value}")

Базис,x1,x2,x3,x4,x5,b
x2,4,5,1,0,0,58
x3,2,7,0,1,0,65
x4,8,3,0,0,1,90
Δ,-3,-4,0,0,0,0


Базис,x1,x2,x3,x4,x5,b
x2,18/7,0,1,-5/7,0,81/7
x1,2/7,1,0,1/7,0,65/7
x4,50/7,0,0,-3/7,1,435/7
Δ,-13/7,0,0,4/7,0,260/7


Базис,x1,x2,x3,x4,x5,b
x0,1,0,7/18,-5/18,0,9/2
x1,0,1,-1/9,2/9,0,8
x4,0,0,-25/9,14/9,1,30
Δ,0,0,13/18,1/18,0,91/2


Оптимальное решение найдено!

Результат:
x1 = 9/2
x2 = 8
x3 = 0
x4 = 0
x5 = 30
Оптимальное значение целевой функции: 91/2
